# Cleaning Mobile Game Data

## 1) Cleaning for google play moblie games

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# read csv
def readcsv(x):
    game_df_raw = pd.read_csv(x)
    return game_df_raw

In [ ]:
game_df_raw = readcsv('gameinfo_raw')

In [ ]:
# Check columns
game_df_raw.columns

In [ ]:
# functions to keep to columns that I want
def dropcln(game_df_raw):
    game_df_raw.drop(game_df_raw.columns.difference(['categories', 'contains_ads', 'downloads_and_revenue', 'has_iap',
       'installs', 'most_popular_country', 'name', 'os', 'price',
       'publisher_country', 'publisher_name', 'rating', 'rating_breakdown',
       'rating_count']), axis=1, inplace=True)
    return game_df_raw

In [ ]:
# excute drop columns function
dropcln(game_df_raw)

In [ ]:
game_df_raw.head()

In [ ]:
# clean the categories and has_iap columns
def remake_columns(game_df_raw):
    game_df_raw['categories'] = game_df_raw['categories'].apply(lambda x: pd.Series(re.split('[,:]',x)))[5]
    game_df_raw['categories'].replace("['']",'', regex=True, inplace=True)
    game_df_raw['has_iap'].replace("r['']",'', regex=True, inplace=True)
    game_df_raw['has_iap'].replace({"'1.0'":1,"True": 1, None:0, float(1.0) :1}, inplace=True)
    return game_df_raw

In [ ]:
# execute function
remake_columns(game_df_raw)

In [ ]:
# set up the "revenue" Column
def convertamount():
    game_df_raw['revenue'] = game_df_raw['downloads_and_revenue'].apply(lambda x: pd.Series(re.split('[{},:]',x)))[4]
    game_df_raw['revenue'].str.strip()
    game_df_raw['revenue'].replace('k', '000', regex=True, inplace=True)
    game_df_raw['revenue'].replace('m', '000000', regex=True, inplace=True)
    game_df_raw['revenue'].replace("[<$'']", '', regex=True, inplace=True)
    game_df_raw['revenue'] = pd.to_numeric(game_df_raw['revenue'])
    return game_df_raw['revenue']

In [ ]:
# execute function
convertamount()

In [ ]:
# set up the "installs" Column
def convertamount3():
    game_df_raw['installs'].str.strip()
    game_df_raw['installs'].replace('[+,]', '', regex=True, inplace=True)
    game_df_raw['installs'] = pd.to_numeric(game_df_raw['installs'])
    return game_df_raw['installs']

In [ ]:
# execute function
convertamount3()

In [ ]:
# set up the "downloadss" Column
def convertamount2():
    game_df_raw['downloads'] = game_df_raw['downloads_and_revenue'].apply(lambda x: pd.Series(re.split('[{},:]',x)))[2]
    game_df_raw['downloads'].replace(r"[<'']", '', regex=True, inplace=True)
    game_df_raw['downloads'].str.strip()
    game_df_raw['downloads'].replace('k', '000', regex=True, inplace=True)
    game_df_raw['downloads'].replace('m', '000000', regex=True, inplace=True)
    game_df_raw['downloads'] = pd.to_numeric(game_df_raw['downloads'])
    return game_df_raw['downloads']

In [ ]:
# execute function
convertamount2()

In [15]:
# eval the column; delect the ''. and turn it into a list
game_df_raw['rating_breakdown'] = game_df_comb['rating_breakdown'].apply(lambda x: eval(x))

In [16]:
# Breaking 'rating_breakdown' feature into 5.
def make5star():
    game_df_comb['one_star'] = game_df_comb['rating_breakdown'].apply(lambda x: x[0])
    game_df_comb['two_star'] = game_df_comb['rating_breakdown'].apply(lambda x: x[1])
    game_df_comb['three_star'] = game_df_comb['rating_breakdown'].apply(lambda x: x[2])
    game_df_comb['four_star'] = game_df_comb['rating_breakdown'].apply(lambda x: x[3])
    game_df_comb['five_star'] = game_df_comb['rating_breakdown'].apply(lambda x: x[4])

In [17]:
# execute function
make5star()

In [19]:
game_df_raw.isna().sum()

categories                0
contains_ads              0
downloads_and_revenue     0
has_iap                   0
installs                  0
most_popular_country      0
name                      0
os                        0
price                     0
publisher_country        81
publisher_name            0
rating                    0
rating_breakdown          0
rating_count              0
revenue                   0
downloads                 0
one_star                  0
two_star                  0
three_star                0
four_star                 0
five_star                 0
dtype: int64

In [ ]:
# rename column os ---> android
game_df_raw.rename(columns={'os':'android'}, inplace=True)

In [ ]:
# replace ios string with 1 for ios columns
game_df_raw['android'] = game_df_raw['android'].replace('android', 1)

In [ ]:
game_df_raw.to_csv('gameinfo_google')

## 2) Cleaning ios Game Data

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
game_df_raw = pd.read_csv("gameinfo_raw_os")

In [ ]:
# functions to keep to columns that I want
def dropcln(game_df_raw):
    game_df_raw.drop(game_df_raw.columns.difference(['categories', 'contains_ads', 'has_iap', 'price', 'rating', 'rating_count', 'rating_breakdown',
       'os', 'publisher_name', 'publisher_country', 'installs', 'most_popular_country','downloads_and_revenue','name']), axis=1, inplace=True)
    return game_df_raw

In [ ]:
dropcln(game_df_raw)

In [ ]:
# Drop na on 'categories' column
game_df_raw['categories'].dropna(how="any", inplace=True)

In [ ]:
# apply eval on 'categories' column
game_df_raw['categories'] = game_df_raw['categories'].apply(lambda x: eval(x))

In [ ]:
# function to split and return the game catergories for each row.
def cat(x):
    v = []
    for g in x:
        if len(g['name'].split(r'/')) > 1:
            v.append(g['name'].split(r'/')[1])
    return v

In [ ]:
# apply cat() function on 'categories' column
game_df_raw['categories'] = game_df_raw['categories'].apply(cat)

In [ ]:
# function to clean "is_iap" columns,set everything to 0 and 1
def iap_remake(game_df_raw):
    game_df_raw['has_iap'].replace("r['']",'', regex=True, inplace=True)
    game_df_raw['has_iap'].replace({"'1.0'":1,"True": 1, None:0, float(1.0) :1}, inplace=True)
    return game_df_raw

In [ ]:
iap_remake(game_df_raw)

In [ ]:
# function to clean "contains_ads" columns,set everything to 0 and 1
def ads_remake(game_df_raw):
    game_df_raw['contains_ads'].replace("r['']",'', regex=True, inplace=True)
    game_df_raw['contains_ads'].replace({"'1.0'":1,"True": 1, None:0, float(1.0) :1}, inplace=True)
    return game_df_raw

In [ ]:
ads_remake(game_df_raw)

In [ ]:
# mask to search the row with incorrect data
mask1 = game_df_raw['rating_breakdown'].apply(lambda x: type(x) != str)

In [ ]:
game_df_raw[mask1]

In [ ]:
# Drop those rows with incorrect data
game_df_raw.drop([464,1128,1215], inplace=True)

In [ ]:
# apply eval on 'downloads_and_revenue' column
game_df_raw['downloads_and_revenue'] = game_df_raw['downloads_and_revenue'].apply(lambda x: eval(x))

In [23]:
# rename column os ---> ios
game_df_raw.rename(columns={'os':'ios'}, inplace=True)

In [24]:
# replace ios string with 1 for ios columns
game_df_raw['ios'] = game_df_raw['ios'].replace('ios', 1)

In [ ]:
def catos(x):
    if len(x) >=1:
        return x[0]
    else:
        return None

In [ ]:
# Further cleaning the 'categories'
game_df_raw['categories'] = game_df_raw4['categories'].apply(catos)

In [ ]:
# Set up revenue column
def convertamount():
    game_df_raw['revenue'] = game_df_raw['downloads_and_revenue'].apply(lambda x: x['revenue'])
    game_df_raw['revenue'].str.strip()
    game_df_raw['revenue'].replace('k', '000', regex=True, inplace=True)
    game_df_raw['revenue'].replace('m', '000000', regex=True, inplace=True)
    game_df_raw['revenue'].replace(r"[<$]", '', regex=True, inplace=True)
    game_df_raw['revenue'] = pd.to_numeric(game_df_raw['revenue'])
    return game_df_raw['revenue']

In [ ]:
convertamount()

In [ ]:
# Set up downloads column
def convertamount2():
    game_df_raw['downloads'] = game_df_raw['downloads_and_revenue'].apply(lambda x: x['downloads'])
    game_df_raw['downloads'].replace(r"[<'']", '', regex=True, inplace=True)
    game_df_raw['downloads'].str.strip()
    game_df_raw['downloads'].replace('k', '000', regex=True, inplace=True)
    game_df_raw['downloads'].replace('m', '000000', regex=True, inplace=True)
    game_df_raw['downloads'] = pd.to_numeric(game_df_raw['downloads'])
    return game_df_raw['downloads']

In [ ]:
convertamount2()

In [ ]:
game_df_raw['rating_breakdown'] = game_df_raw['rating_breakdown'].apply(lambda x: eval(x))

In [ ]:
# break 'rating_breakdown' into 5 columns
def make5star():
    game_df_raw['one_star'] = game_df_raw['rating_breakdown'].apply(lambda x: x[0])
    game_df_raw['two_star'] = game_df_raw['rating_breakdown'].apply(lambda x: x[1])
    game_df_raw['three_star'] = game_df_raw['rating_breakdown'].apply(lambda x: x[2])
    game_df_raw['four_star'] = game_df_raw['rating_breakdown'].apply(lambda x: x[3])
    game_df_raw['five_star'] = game_df_raw['rating_breakdown'].apply(lambda x: x[4])

In [ ]:
make5star()

In [ ]:
game_df_raw.to_csv('gaminfo_os')

## Combine csv/dataframe ios and google

In [ ]:
# read 2 dataframe google and ios
game_df_and.to_csv('gaminfo_google.csv')
game_df_ios.to_csv('gaminfo_ios.csv')

In [ ]:
# combine dataframe
game_df_comb = pd.concat([game_df_and, game_df_ios], axis=0)

In [47]:
game_df_comb['ios'].fillna(value=0, inplace=True)

In [47]:
game_df_comb['android'].fillna(value=0, inplace=True)